This notebook seeks to explore the gender diversity of the different apache projects & the process

In [5]:
from pyspark import *
from pyspark.sql import *
from pyspark.sql.functions import explode
from pyspark.sql.types import *

import json
import os
import meetup.api
from copy import copy
import time

from watson_developer_cloud import ToneAnalyzerV3

API key configuration

In [6]:
meetup_key = os.getenv("MEETUP_APIKEY")
tone_bluemix_user = os.getenv("TONE_BLUEMIX_USERNAME")
tone_bluemix_password = os.getenv("TONE_BLUEMIX_PASSWORD")

In [2]:
session = SparkSession.builder.appName("whatCanWeLearnFromTheSixties").getOrCreate()
sc = session.sparkContext

The first thing we want to get is the committers and PMC members, this information is stored in LDAP but also available in JSON. Eventually we will want to enrich this with mailing list information

In [105]:
def loadFlatJsonFile(path, explodeKey, schema=None):
    """Load a flat multi-line json file and convert into Spark & explode"""
    rdd = sc.wholeTextFiles(path).values()
    df = (session.read.schema(schema)
            .json(rdd))
    return df.select(explode(explodeKey))

In [106]:
apache_people_schema = StructType([StructField("lastCreateTimestamp", StringType()),
                     StructField("people",
                                 MapType(StringType(), MapType(StringType(), StringType())))])
apache_people_df = loadFlatJsonFile(path="http_data_sources/public_ldap_people.json", # http://people.apache.org/public/public_ldap_people.json
                                 explodeKey="people", schema=apache_people_schema)

In [213]:
def project_on_github(project):
    """Returns if a project is on github"""
    import urllib3
    http = urllib3.PoolManager()
    r = http.request('GET', "https://github.com/apache/{0}".format(project))
    return r.status == 200
session.catalog.registerFunction("on_github", project_on_github, BooleanType())
# Except I'm a bad person so....
from pyspark.sql.catalog import UserDefinedFunction
project_on_github_udf = UserDefinedFunction(project_on_github, BooleanType(), "on_github")
session.catalog._jsparkSession.udf().registerPython("on_github", udf._judf)

In [216]:
apache_committees_schema = StructType([StructField("lastCreateTimestamp", StringType()),
                     StructField("committees",
                                 MapType(StringType(), MapType(StringType(), StringType())))])
apache_committees_df = loadFlatJsonFile(path="http_data_sources/public_ldap_committees.json", # http://people.apache.org/public/public_ldap_people.json
                                 explodeKey="committees", schema=apache_committees_schema)
apache_committees_on_github_df = apache_committees_df.filter(project_on_github_udf(apache_committees_df.key))
committee_names_df = apache_committees_on_github_df.select(apache_committees_df.key.alias("project"))
committee_names_df.cache()
committee_names_df.count()

[Row(project='abdera'),
 Row(project='accumulo'),
 Row(project='ace'),
 Row(project='activemq'),
 Row(project='airavata'),
 Row(project='allura'),
 Row(project='ambari'),
 Row(project='ant'),
 Row(project='any23'),
 Row(project='apr'),
 Row(project='archiva'),
 Row(project='aries'),
 Row(project='arrow'),
 Row(project='asterixdb'),
 Row(project='aurora'),
 Row(project='avro'),
 Row(project='bahir'),
 Row(project='beam'),
 Row(project='bigtop'),
 Row(project='bloodhound'),
 Row(project='bookkeeper'),
 Row(project='brooklyn'),
 Row(project='buildr'),
 Row(project='bval'),
 Row(project='calcite'),
 Row(project='camel'),
 Row(project='cassandra'),
 Row(project='cayenne'),
 Row(project='celix'),
 Row(project='chemistry'),
 Row(project='chukwa'),
 Row(project='clerezza'),
 Row(project='climate'),
 Row(project='cloudstack'),
 Row(project='cocoon'),
 Row(project='couchdb'),
 Row(project='crunch'),
 Row(project='ctakes'),
 Row(project='curator'),
 Row(project='cxf'),
 Row(project='deltacloud'),

Attempt to fetch relevant past & present meetups for each project - idea based on the listing at https://www.apache.org/events/meetups.html but different code

In [200]:
logger = logging.getLogger()
logger.setLevel("WARN")
def lookup_relevant_meetup(project_name):
    """Lookup relevant meetups for a specific project."""
    import logging
    import time
    import meetup.api
    logger = logging.getLogger()
    meetup_delay = 30
    meetup_reset_delay = 1800 # 30 minutes
    standard_keys = {"text_format": "plain", "trending": "desc=true", "and_text": "true", "city": "san francisco", "country": "usa", "text": "apache " + project_name, "radius": 10000}
    results = {"upcoming": [], "past": []}
    for status in ["upcoming", "past"]:
        keys = copy(standard_keys)
        keys["status"] = status
        count = 200
        base = 0
        while (count == 200):
            logging.debug("Fetch {0} meetups for {1} on base {2}".format(status, project_name, base))
            project_name = "spark"
            client = client = meetup.api.Client(meetup_key)
            if base > 0:
                keys["page"] = base
            # Manually sleep for meetup_reset_delay on failure, the meetup-api package retry logic sometimes breaks :(
            response = None
            retry_count = 0
            while response is None and retry_count < 10:
                try:
                    response = client.GetOpenEvents(**keys)
                except:
                    response = None
                    retry_count += 1
                    time.sleep(meetup_reset_delay)
                    try:
                        response = client.GetOpenEvents(**keys)
                    except:
                        response = None
            try:
                count = response.meta['count']
                base = base + count
                results[status].append(response.results)
                time.sleep(meetup_delay)
            except:
                count = 0
    return (project_name, results)

project_meetups_df = committee_names_df.rdd.map(lambda x: x.project).map(lookup_relevant_meetup)

In [201]:
project_meetups_df.cache()
project_meetups_df.take(1)

KeyboardInterrupt: 

For the provided projects attempt to lookup their GitHub

In [203]:
def lookup_project_git(project):
    """Returns the project github for a specific project. Assumes project is git hosted"""
    return "git://git.apache.org/{0}".format(project)
    

In [221]:
def fetch_project_github_data(project):
    from perceval.backends.core.github import GitHub as perceval_github
    gh_backend = perceval_github(owner="apache", repository=project)
    return list(gh_backend.fetch())

In [222]:
def fetch_project_git_data(project):
    project_git = lookup_project_git(project)
    from perceval.backends.core.git import Git as perceval_git
    git_backend = perceval_git(owner="apache", repository=project)
    return list(gh_backend.fetch())

In [29]:
def lookup_crunchbase_info(people_and_projects):
    """Lookup a person a crunch base and see what the gender & company is.
    Filter for at least one mention of their projects."""
    from bs4 import BeautifulSoup
    from selenium import webdriver
    import re
    driver = webdriver.Firefox()
    for (name, projects) in people_and_projects:
        # robots.txt seems to be ok with person for now, double check before re-running this
        url = "https://www.crunchbase.com/person/{0}".format(name.replace(" ", "-"))
        try:
            driver.get(url)
            text = driver.page_source
            lower_text = text.lower()
            if any(project.lower() in lower_text for project in projects):
                soup = BeautifulSoup(text, "html.parser")
                stats = soup.findAll("div", { "class" : "overview-stats" })
                # Hacky but I'm lazy
                result = {}
                try:
                    m = re.search("Gender:\</dt\>\<dd\>(<gender>.+?)\<", text)
                    result["gender"] = m.group('gender')
                except:
                    # If nothing matches thats ok
                    pass
                try:
                    m = re.search("data-name=\"(<company_name>.+?)\" data-permalink=\"\/organization", text)
                    result["company"] = m.group('company_name')
                except:
                    # No match no foul
                    pass
                return [result]
            else:
                return None
        except:
            return None

In [ ]:
result = lookup_crunchbase_info([("holden karau", ["spark"])])
result

In [295]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(result.page_source, "html.parser")

In [311]:
str(soup.findAll("div", { "class" : "info-card-overview-content"})[0])

'<div class="info-card-overview-content container" id="info-card-overview-content"><div class="definition-list-container"><dl class="definition-list container"><div class="overview-stats"><dt>Primary Role</dt><dd>Principal Software Engineer <span class="connecting">@</span> <a class="follow_card" data-follower-count="1346" data-image="https://crunchbase-production-res.cloudinary.com/image/upload/c_pad,h_98,w_98/v1488043438/rvdqo4tfkivzmgvpkc36.png" data-name="IBM" data-permalink="/organization/ibm" data-type="organization" data-uuid="fc5fbe7a562dd577b05e46797caa134c" href="/organization/ibm">IBM</a></dd></div><dt>Gender:</dt><dd>Female</dd><dt>Location:</dt><dd>San Francisco</dd><dt>Social: </dt><dd class="social-links"><a class="icons twitter" data-icons="twitter" href="https://twitter.com/holdenkarau" rel="nofollow" target="_blank"></a><a class="icons linkedin" data-icons="linkedin" href="https://www.linkedin.com/in/holdenkarau" rel="nofollow" target="_blank"></a></dd></dl><!--end .d

One of the things that is interesting is understanding what the tones of the meetup descriptions & mailing list posts are. We can use https://www.ibm.com/watson/developercloud/tone-analyzer/api/v3/?python#introduction

In [14]:
def lookup_tone(document):
    """Looks up the tone for a specific document. Returns a json blob."""
    from watson_developer_cloud import ToneAnalyzerV3
    tone_analyzer = ToneAnalyzerV3(
        username=tone_bluemix_user,
        password=tone_bluemix_password,
        version='2016-05-19 ')
    return tone_analyzer.tone(text=document)

In [13]:
lookup_tone("oh no you didn't girl")

{'document_tone': {'tone_categories': [{'category_id': 'emotion_tone',
    'category_name': 'Emotion Tone',
    'tones': [{'score': 0.029733, 'tone_id': 'anger', 'tone_name': 'Anger'},
     {'score': 0.11531, 'tone_id': 'disgust', 'tone_name': 'Disgust'},
     {'score': 0.061847, 'tone_id': 'fear', 'tone_name': 'Fear'},
     {'score': 0.218219, 'tone_id': 'joy', 'tone_name': 'Joy'},
     {'score': 0.657275, 'tone_id': 'sadness', 'tone_name': 'Sadness'}]},
   {'category_id': 'language_tone',
    'category_name': 'Language Tone',
    'tones': [{'score': 0.0,
      'tone_id': 'analytical',
      'tone_name': 'Analytical'},
     {'score': 0.0, 'tone_id': 'confident', 'tone_name': 'Confident'},
     {'score': 0.0, 'tone_id': 'tentative', 'tone_name': 'Tentative'}]},
   {'category_id': 'social_tone',
    'category_name': 'Social Tone',
    'tones': [{'score': 0.020611,
      'tone_id': 'openness_big5',
      'tone_name': 'Openness'},
     {'score': 0.421959,
      'tone_id': 'conscientiousne